In [1]:
import matplotlib.pyplot as plt
import numpy as np
from math import sqrt
import os

from src.util import generate_node_cargo_size_change, calculate_single_truck_deviation, ConsoleLogger
from src.pdpt_ini_sol import solve_pdotw_mip
import pickle
from pathlib import Path

dir_ = '/home/tan/Documents/GitHub/pdpt_2022/toy'

In [ ]:
filename = dir_ + '/toy_initSol.txt'
pdpt_ins = read_pdpt_pickle(filename, verbose = verbose-1) 


truck_yCycle_file, truck_used_file, truck_route_file, \
cargo_route_file, S_sol_file, A_sol_file, D_sol_file, \
Sb_sol_file, Ab_sol_file, Db_sol_file = read_route_solution_PDPT(filename, verbose = 0)